In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
url = "https://raw.githubusercontent.com/4GeeksAcademy/regularized-linear-regression-project-tutorial/main/demographic_health_data.csv"
df = pd.read_csv(url)
target = "Heart disease_number"

In [3]:
df.drop(columns=['STATE_NAME','COUNTY_NAME'], inplace= True)

In [4]:
X = df.drop(columns=target).copy()
y = df[target]

In [5]:
df

,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,...,COPD_number,diabetes_prevalence,diabetes_Lower 95% CI,diabetes_Upper 95% CI,diabetes_number,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code
0,1001,55601,6787,12.206615,7637,13.735364,6878,12.370281,7089,12.749771,...,3644,12.9,11.9,13.8,5462,3.1,2.9,3.3,1326,3
1,1003,218022,24757,11.355276,26913,12.344167,23579,10.814964,25213,11.564429,...,14692,12.0,11.0,13.1,20520,3.2,3.0,3.5,5479,4
2,1005,24881,2732,10.980266,2960,11.896628,3268,13.134520,3201,12.865239,...,2373,19.7,18.6,20.6,3870,4.5,4.2,4.8,887,6
3,1007,22400,2456,10.964286,2596,11.589286,3029,13.522321,3113,13.897321,...,1789,14.1,13.2,14.9,2511,3.3,3.1,3.6,595,2
4,1009,57840,7095,12.266598,7570,13.087828,6742,11.656293,6884,11.901798,...,4661,13.5,12.6,14.5,6017,3.4,3.2,3.7,1507,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3135,56037,43051,6104,14.178532,6326,14.694200,5359,12.448027,6577,15.277229,...,2098,8.9,8.3,9.6,2834,2.6,2.4,2.8,821,5
3136,56039,23081,2384,10.328842,2185,9.466661,2967,12.854729,4093,17.733200,...,928,7.2,6.5,8.0,1360,2.4,2.2,2.6,447,5
3137,56041,20299,3121,15.375142,3205,15.788955,2153,10.606434,2702,13.311001,...,1163,10.4,9.5,11.2,1500,3.0,2.8,3.2,430,5
3138,56043,7885,858,10.881420,1113,14.115409,715,9.067850,903,11.452124,...,506,11.3,10.3,12.1,686,3.4,3.2,3.7,207,6


In [6]:
def is_binary(df_, nums):
    df = df_.copy()
    variables = []
    for var in nums:
        flag = True
        unique = df_[var].unique()
        for value in unique:
            if value not in [0, 1, np.nan, 0.0, 1.0]:
                flag = False
        if flag == True:
            variables.append(var)
    return variables


def breakdown_vars(df, off_binary=False):
    """
    This function allow us categorize accodign to numerical or not
    """
    binaries = is_binary(df, df.columns)
    categorial = []
    nonormal = []
    normal = []
    for t in df.columns:
        if off_binary == False:
          if (df[t].dtypes.name=="object" or df[t].dtypes.name=='category') and  t not in binaries:
            categorial.append(t)
        else:
           if (df[t].dtypes.name=="object" or df[t].dtypes.name=='category'):
            categorial.append(t)
        if (df[t].dtypes=="int64" or df[t].dtypes=="float64") and t not in binaries:
                n,p = stats.shapiro(df[t])
                if p<0.05:
                    nonormal.append(t)
                else:
                    normal.append(t)
    if off_binary == False:
      return categorial, binaries, nonormal, normal
    else:
      return categorial, nonormal, normal

cat, nonormal ,  normal = breakdown_vars(df,  off_binary=True)

In [7]:
variables=[]
for var in X.columns:
  if var in nonormal + normal:
    corr = np.corrcoef(y, df[var])[0][1]
    if abs(corr) > 0.80:
      variables.append(var)


In [8]:
df=df[variables]

In [9]:
df.drop(columns=['TOT_POP','Total Population','Population Aged 60+', 'county_pop2018_18 and older','POP_ESTIMATE_2018','GQ_ESTIMATES_2018'],inplace=True)

/tmp/ipykernel_694/2073105899.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['TOT_POP','Total Population','Population Aged 60+', 'county_pop2018_18 and older','POP_ESTIMATE_2018','GQ_ESTIMATES_2018'],inplace=True)


In [10]:
X = df
y = y

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y, shuffle=True,test_size=0.20, random_state=12)

In [12]:
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.model_selection import KFold

In [13]:
def grid_lasso(X_train,y_train):
  model= linear_model.Lasso(random_state=12)
  alpha= np.array([0.001,0.1,1,10])
  grid=dict(alpha=alpha)
  cv = KFold (n_splits = 5, shuffle=True, random_state=10)
  grid_search=GridSearchCV(estimator=model,
                           param_grid=grid,
                           cv=cv,
                           scoring=mean_squared_error)
  grid_result=grid_search.fit(X_train,y_train)
  return grid_search.best_estimator_

In [14]:
clf=grid_lasso(X_train,y_train)

/home/vscode/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.179e+08, tolerance: 6.207e+07
  model = cd_fast.enet_coordinate_descent(
/home/vscode/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/vscode/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/home/vscode/.local/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 191, in wrapper
    params = func_sig.bind(*args, **kwargs)
  File "/usr/local/lib/python3.10/inspect.py", line 318

In [15]:
preds=clf.predict(X_test)

In [16]:
print(mean_squared_error (y_test,preds))
print(r2_score(y_test, preds))

286873.41492429166
0.9980330113287822


In [17]:
!pip install deap

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 3.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [18]:
import random
import deap
from deap import creator, base, tools, algorithms

In [19]:
def score(solution,classifier,X_train,y_train,metric):
  X_train, y_train= X_train.copy(), y_train.copy()
  X_train, X_val, y_train, y_val= train_test_split(X_train, y_train,
                                                   test_size=0.30, shuffle=True,
                                                   random_state=12)
  
  solution= np.array(solution, dtype='bool')
  X_train= X_train.loc[:,solution].to_numpy()
  X_val=X_val.loc[:,solution].to_numpy()
  return metric(y_val, classifier(X_train, y_train).predict(X_val)),

In [20]:
def feature_ga(classifier,X_train, y_train, metric=mean_squared_error):
  random.seed(12)
  chromosomal_size= X_train.shape[1]
  creator.create("FitnessMax", base.Fitness, weights=(-1.0,))
  creator.create('Individual', list, fitness=creator.FitnessMax)
  toolbox= base.Toolbox()
  toolbox.register('binaryGen', random.randint, 0,1)
  toolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.binaryGen, chromosomal_size)
  toolbox.register('population', tools.initRepeat, list, toolbox.individual, 5)
  toolbox.register('evaluate', lambda x: score(x, classifier, X_train, y_train, metric))
  toolbox.register('mate', tools.cxOnePoint)
  toolbox.register('mutate', tools.mutFlipBit, indpb=0.05)
  toolbox.register('select', tools.selTournament, tournsize=3)
  pop = toolbox.population()
  pop, logbox = algorithms.eaSimple(pop, toolbox, cxpb=0.35, mutpb=0.05, ngen=2, verbose=False)
  return pop[0], toolbox.evaluate(pop[0])

In [21]:
feature_ga(grid_lasso, X_train, y_train)

/home/vscode/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.856e+08, tolerance: 4.904e+07
  model = cd_fast.enet_coordinate_descent(
/home/vscode/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/vscode/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/home/vscode/.local/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 191, in wrapper
    params = func_sig.bind(*args, **kwargs)
  File "/usr/local/lib/python3.10/inspect.py", line 318

([0,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  1],
 (np.float64(594071.6288419659),))

In [22]:
sol=[0,1,1,0,0,0,1,1,0,1,0,1,1,0,1,0,1,1,1,0,0,0,0,0,0,1,0,1,0,1,0,1]

In [23]:
seleccion=[bool(s) for s in sol]
X_train=X_train.loc[:,seleccion]
X_test=X_test.loc[:,seleccion]

In [24]:
clf=grid_lasso(X_train,y_train)

/home/vscode/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.758e+08, tolerance: 6.207e+07
  model = cd_fast.enet_coordinate_descent(
/home/vscode/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/vscode/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/home/vscode/.local/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 191, in wrapper
    params = func_sig.bind(*args, **kwargs)
  File "/usr/local/lib/python3.10/inspect.py", line 318

In [25]:
preds3=clf.predict(X_test)

In [26]:
print(mean_squared_error (y_test,preds))
print(r2_score(y_test, preds3))

286873.41492429166
0.9979855212984899


In [27]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_train_= scaler.fit_transform(X_train)
X_test_ = scaler.transform(X_test)

In [28]:
clf=grid_lasso(X_train_,y_train)

/home/vscode/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.759e+08, tolerance: 6.207e+07
  model = cd_fast.enet_coordinate_descent(
/home/vscode/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/vscode/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/home/vscode/.local/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 191, in wrapper
    params = func_sig.bind(*args, **kwargs)
  File "/usr/local/lib/python3.10/inspect.py", line 318

In [29]:
preds2=clf.predict(X_test_)

In [30]:
print(mean_squared_error (y_test,preds2))
print(r2_score(y_test, preds2))

293783.00106434495
0.997985634761442
